# Deep RecSys Course
## Домашнее задание 1

### ФИО: <впишите>

### Введение
В этом домашнем задании вы полностью пройдёте базовый пайплайн: подготовка данных → метрики → несколько рекомендательных подходов → итоговый лидерборд.

### Используемые библиотеки
В данном задании потребуются следующие библиотеки:
* [polars](https://pola.rs/) - библиотека для работы с данными (человечество постепенно уходит от `pandas`'а)
* [implicit](https://github.com/benfred/implicit) - библиотека для обучения и применения различных коллаборативных рекомендательных моделей
* [torch](https://pytorch.org/) - no comments
* [gensim](https://radimrehurek.com/gensim/) - обучение **word2vec**

### Данные
Данные лежат в архиве `data.zip`, который состоит из:
* `interactions.parquet` - user-item взаимодействия из датасета Yambda (лайки для 500m версии)
* `embeddings.parquet` - уже пофильтрованные и чуть более плотно запакованные эмбеддинги треков из Yambda
* `artists.parquet` - метаданные айтемов с маппингом в артистов

Скачать архив можно здесь: [ссылка на google disk](https://drive.google.com/file/d/1PojPVpXGBAqzHQi97QAFhJ9gnPsXxveS/view?usp=sharing). В следующем блоке мы в любом случае скачиваем датасет, поэтому самостоятельно его можно не качать.

### Guidelines
- Для выполнения ДЗ достаточно использовать Google Collab с T4
- Детерминизм: фиксируйте сиды там, где это важно
- Не используйте данные из теста при обучении/подготовке моделей
- Тест — **последняя неделя** (по timestamp), как описано ниже
- После каждого этапа запускайте проверки внутри ноутбука
- Старайтесь избегать работы с сырыми питоновскими объектами (словарями, списками, интами) там, где можно применить методы из `polars` - они будут в десятки-сотни раз быстрее и читабельней
- Чтобы быстрее обнаруживать, что код написан неоптимально и выполняется слишком долго, старайтесь оборачивать циклы в `tqdm` и выводить progress bar
- Во время отладки кода можно посэмплировать данные для скорости с помощью `data.sample(fraction=0.1`. Для проверки тестов после отладки надо сделать запуски с полным датасетом

### Разбалловка
1. Подготовка данных (1 балл)
2. Оценка качества (1 балл)
3. Топ популярного (1 балл)
4. Рекомендации по артистам (1 балл)
5. Item-to-Item рекомендации (2 балла)
6. Item2Vec (1 балл)
7. Item-based Collaborative Filtering (1 балл)
8. ALS (1 балл)
9. Вопросы (1 балл)

Суммарно - 10 баллов.

In [1]:
import tests

!pip install gensim
# implicit устанавливается долго, минут 10
!pip install implicit

!pip install -q gdown
!gdown --id 1PojPVpXGBAqzHQi97QAFhJ9gnPsXxveS -O dataset.zip
!unzip -q dataset.zip

### 1. Подготовка данных (1 балл)

**Задача:**
1) Считать данные (взаимодействия, эмбеддинги, метаданные).  
2) Оставить только взаимодействия, для которых есть эмбеддинги.  
3) Поджоинить метаданные (артисты треков) ко всем взаимодействиям.  
4) Сделать core фильтрацию: оставить только айтемы с ≥5 взаимодействиями (почему мы это делаем -- исключительно для удобства и скорости, в реальной работе так делать не стоит)
5) Сделать train-test split: последнюю неделю положить в тест.  
6) Ограничить тест юзерами, у которых есть взаимодействия в трейне.  
7) Подготовить для оценки качества `test_targets: Dict[uid, List[item_id]]`.

После этого блока должны существовать `train`, `test`, `embeddings`, `artists`, `test_targets`.

Для этого блока полезны как минимум следующие методы:
* `pl.read_parquet` - для чтения данных
* `.filter, .value_counts` помогут сделать core-фильтрацию
* `df.join(...)` - при джойне метаданных надо использовать `how='left'`, а не `how='inner'`
* `df.join(other, on=some_key, how='semi')` -- режим `semi` используется для фильтраций (оставить только те строки из исходного датафрейма, ключ которых присутствует во второй таблице)

In [ ]:
from typing import Dict, List, Tuple, Any, Optional
from collections import defaultdict
import os

import numpy as np
import polars as pl

import tests

# Пути к данным (ожидается, что они лежат рядом с ноутбуком)
DATA_DIR = "."
PATH_INTERACTIONS = os.path.join(DATA_DIR, "interactions.parquet")
PATH_EMBEDDINGS = os.path.join(DATA_DIR, "embeddings.parquet")
PATH_ARTISTS = os.path.join(DATA_DIR, "artists.parquet")

# Глобальные параметры
TOPK = 100
CORE_MIN_INTERACTIONS_PER_ITEM = 5
TEST_INTERVAL_SECONDS = 7 * 24 * 60 * 60

# Для воспроизводимости
np.random.seed(42)

data = pl.read_parquet(PATH_INTERACTIONS)
embeddings = pl.read_parquet(PATH_EMBEDDINGS)
artists = pl.read_parquet(PATH_ARTISTS)

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [3]:
# Запуск автопроверок (для них необходим файл tests.py)

tests.check_data_split(train=train, test=test, embeddings=embeddings, artists=artists, test_targets=test_targets)

### 2. Оценка качества (1 балл)

#### 2.1 Определения метрик

2.1.1 Пусть для пользователя $u$:

* $G_u \subset \mathcal{I}$ — множество релевантных айтемов (ground truth)
* $R_u = (r_{u,1}, \dots, r_{u,K})$ — упорядоченный список рекомендаций длины $K$

Обозначим индикатор релевантности $I_{u,k} = [ r_{u, k} \in G_u]$. В простонародье его еще часто называют `hits`.

2.1.2 **Hitrate@K** равен единичке, если мы угадали в topK хотя бы один релевантный айтем:
* $
\text{Hitrate@K} = \frac{1}{|U|}
\sum_{u \in U}
\left[ \sum_{k=1}^{K} I_{u,k} > 0 \right]
$

2.1.3 **Recall@K** оценивает долю угаданных релевантных айтемов (от всех релевантных айтемов):
* $
\text{Recall@K} = \frac{1}{|U|}
\sum_{u \in U}
\frac{
\sum_{k=1}^{K} I_{u,k}
}{
\min(|G_u|, K)
}
$

2.1.4 Для подсчета **nDCG@K** нужно сначала посчитать **DCG@K**, затем посчитать **iDCG@K** (DCG в случае идеального ранжирования), затем одно поделить на другое:
* $
\text{DCG@K}(u) = \sum_{k=1}^{K}
\frac{I_{u,k}}{\log_2(k+1)}
$
* $
\text{iDCG@K}(u) = \sum_{k=1}^{\min(|G_u|,K)}
\frac{1}{\log_2(k+1)}
$
* $
\text{nDCG@K} = \frac{1}{|U|}
\sum_{u \in U}
\frac{\text{DCG@K}(u)}{\text{iDCG@K}(u)}
$

2.1.5 **Coverage@K** - это число уникальных айтемов во всех рекомендациях, деленное на размер каталога:

* $
\text{Coverage@K} = \frac{|\bigcup_{u \in U} R_u|}{|\mathcal{I}_{train}|},
$ где $\mathcal{I}_{train}$ — каталог айтемов в train.
* в качестве размера каталога используем количество айтемов, которые нам доступны для рекомендации на момент рекомендации (то есть количество уникальных айтемов в `train`)

#### 2.2 Что нужно сделать
Реализуйте функции:
- `get_metrics(targets, candidates, topk) -> dict(hitrate, recall, ndcg)`
- `evaluate(targets_by_user, candidates_by_user, catalog_size, topk) -> dict(hitrate, recall, ndcg, coverage)`

**Важно:** `candidates[uid]` должен иметь длину ровно `topk` (и без `None`).  


In [4]:
def get_metrics(targets: List[int], candidates: List[int], topk: int) -> Dict[str, float]:
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass


def evaluate(
    targets: Dict[int, List[int]],
    candidates: Dict[int, List[int]],
    catalog_size: int,
    topk: int = 100,
) -> Dict[str, float]:
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

In [5]:
tests.check_metrics(get_metrics=get_metrics, evaluate=evaluate)

### 3. Топ популярного (1 балл)

Сделайте топ популярных айтемов по train взаимодействиям и посчитайте метрики с помощью `evaluate`.

Полезные методы из `polars`: `.value_counts, .sort, .head, .to_numpy, .tolist, .n_unique`

In [2]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [7]:
# нужно сложить результат метода evaluate в словарь metrics_toppop, далее в ноутбуке аналогично
tests.check_top_pop(metrics_toppop)

### 4. Рекомендации по артистам (1 балл)

Что хотим:
1. Взять последние N лайков пользователя (для моделей и всех подсчетов нужно использовать ТОЛЬКО `train`, тест используется исключительно для оценки качества)
2. Посчитать по ним любимых артистов пользователя (отсортировать по количеству лайков у артиста)
3. Взять у любимых артистов их самые популярные треки (чтобы посчитать популярность треков, нужно использовать train)
4. Оставить те, которые пользователь еще не видел (не лайкал)
5. Их порекомендовать

Фактически мы формируем рекомендации по счётчикам - учитываем "сколько раз пользователь лайкал артиста" и "сколько раз трек артиста был лайкнут".

В этом методе не для всех пользователей найдётся 100 кандидатов, поэтому нужно дополнить рекомендации до 100 каким-то другим методом, то есть сделать fallback. В данном случае предлагается делать fallback с помощью top_pop:
* если у пользователя набралось меньше 100 рекомендаций, то идём по top_pop и добавляем кандидатов в рекомендации, пока не заполним до 100
* при этом добавляем только unseen треки (те треки, которых у пользователя еще не было)

In [ ]:
# именно такие значения ожидает автопроверка tests.check_artist_recs
N_LAST_EVENTS = 100  # n последних взаимодействий
PER_ARTIST_LIMIT = 20  # берем для рекомендаций 20 айтемов из каждого любимого артиста


# эту функцию будем переиспользовать в следующих пунктах
def fallback_to_toppop(cands_by_uid: Dict[int, List[int]], top_pop: np.ndarray, topk: int) -> Dict[int, List[int]]:
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass


###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [9]:
tests.check_artist_recs(metrics_artist)

### 5. Item-to-Item рекомендации (2 балла)

Здесь предлагается реализовать классический item-to-item алгоритм:
1. Считаем для каждого айтема список похожих айтемов
2. Берём последние N взаимодействий пользователя, для каждого вытаскиваем список похожих
3. Агрегируем кандидатов из списка похожих, учитывая суммарные похожести (если айтем встретился в двух списках кандидатов у пользователя, суммируем похожесть из каждого); при этом отфильтровываем все, что уже было у пользователя в истории
4. Оставляем только topk кандидатов

Реализация должна состоять из двух функций:
1) `get_similar_items_gpu(item_ids, item_embeddings, topk_sim)` - для каждого айтема находим top similar items по cosine. 
* используем pytorch (и GPU) - иначе подсчет будет очень долгим
* создаем тензор с эмбеддингами, переводим на GPU, $l2$-нормализуем
* идём батчами по эмбеддингам, для каждого батча с помощью `torch.topk` считаем честный топ похожих; бачти -- это важно! сильно влияет на скорость
* запоминаем эту информацию в словаре вида `item_id -> [(cand_item_id, similarity), ...]`

2) `get_candidates_item2item(interactions, similar_items, ...)` - для каждого пользователя агрегируем похожести по последним N взаимодействиям.
* нужно буквально реализовать выше описанный алгоритм (берём последние N взаимодействий, суммируем похожести всех полученных похожих айтемов)
* для написания этой функции лучше не пытаться сделать супер оптимальный код через `polars` (у него очень большое пиковое потребление оперативной памяти), а как раз манипулировать словарями, списками, etc
* для ускорения предлагается использовать `heapq.nlargest` для поиска `topk` элементов среди уже подсчитанных суммарных похожестей (раза в полтора-два быстрее, чем делать полную сортировку всех кандидатов со всех списков)

В пункте 2 нужно дополнительно поддержать time decay:
* свежие взаимодействия должны вносить более высокий вклад, поэтому при суммировании похожестей для конкретного кандидата мы учитываем свежесть события, из списка которого берём похожесть
* вес события $2^{-\frac{L-t}{\tau}}$, где $L$ - длина истории пользователя, $t$ - позиция события в истории (начиная с единички), $\tau$ - период полураспада, то есть насколько быстро затухает сигнал от событий. Например, при $\tau = 10$ вес события падает в два раза, если оно идет на 10 позиций раньше в истории
* Получаем $\text{score}(u, j) =
\sum_{t=1}^{L}
2^{- \frac{L - t}{\tau}}
\cdot
s(i_{u,t}, j)
$

**Важно:** при использовании метода `get_similar_items_gpu` набираем не 100 кандидатов, а в два раза больше (2 * topk), чтобы после фильтрации по истории пользователя и слияния всех списков у нас было больше шансов набрать `topk`.

Полезные методы из `polars`: `.to_torch()`, `.to_numpy()` - позволяют сразу получить тензоры для айдишников / эмбеддингов

In [10]:
import torch
import torch.nn.functional as F
import heapq

def get_similar_items_gpu(
    item_ids: np.ndarray,
    item_embeddings: np.ndarray,
    block: int = 1024,
    topk: int = 200,
    device: str = "cuda",
) -> Dict[int, List[Tuple[int, float]]]:
    """
    Возвращает словарь item_id -> [(cand_item_id, similarity), ...] длины topk
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass


def get_candidates_item2item(
    interactions: pl.DataFrame,
    similar_items: Dict[int, List[Tuple[int, float]]],
    n_last: int = 30,
    half_life_frac: float = 0.5,
    topk: int = 100,
) -> Dict[int, List[int]]:
    """
    half_life_frac интерпретируется как доля n_last: half_life = half_life_frac * n_last
    w(r) = 2^{-r / half_life}, где r=0 для самого последнего события.
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

In [3]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [12]:
tests.check_i2i_recs(metrics_i2i)

### 6. Item2Vec (1 балл)

Item-to-item можно использовать с буквально любыми похожестями айтемов.

Теперь давайте попробуем сами обучить эмбеддинги айтемов, посчитать похожести и опять применить item-to-item.

Будем использовать подход Item2Vec - применяем подход word2vec к последовательностям айтемов вместо последовательностей слов.

Для этого предлагается использовать `gensim`:
* нужно создать `corpus` из списка списков строк вида `[['1', '2'], ['3', '4', '5']]`, в котором строки - это буквально строки айдишников айтемов, а внутренние списки - это сгруппированные взаимодействия пользователя (хронологически отсортированные)
* Вызвать метод `gensim.models.Word2Vec`. Предлагаемые настройки `vector_size=100, window=5, min_count=1, sg=0, epochs=5`

После обучения, эмбеддинги можно достать с помощью `w2v.wv.vectors` и соответствующие айдишники с помощью `w2v.wv.index_to_key`.

Далее предлагается применить пайплайн из прошлого пункта: `get_similar_items_gpu` + `get_candidates_item2item`

In [4]:
from gensim.models import Word2Vec

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [14]:
tests.check_w2v_recs(metrics_w2v)

### 7. Item-based Collaborative Filtering (1 балл)

Теперь попробуем применить тот же item-to-item подход, но используя в качестве векторов большие разреженные векторы из user-item матрицы. Для подсчета разреженных близостей наш GPU пайплайн не подходит (не умеет работать с разреженными данными), поэтому будем использовать библиотеку `implicit`

1. Сначала нужно собрать CSR user-item матрицу из наших `train` взаимодействий с помощью `scipy.sparse.csr_matrix`
* предлагается использовать формат вида `csr_matrix((ones, (user_ids, item_ids)), shape=(num_users, num_items))`
* чтобы просуммировать взаимодействия с одними и теми же айтемами можно использовать `.sum_duplicates`
* также понадобится от исходных айдишников (которые необязательно от 1 до `num_items / num_users`) перейти к компактным айдишникам, сделав маппинг {old_item_id: new_item_id}
* простой вариант -- сделать это с помощью словаря, более быстрый - использовать метод вида `train.select("uid").unique().with_row_index()` (аналогично для айтемов)

2. Чтобы с помощью `implicit` получить списки похожих, нужно использовать комбинацию из `CosineRecommender.fit`, и `.similar_items`

3. Чтобы сформировать рекомендации, используем нашу функцию `get_candidates_item2item`

**Warning:** неаккуратно написанный код в этом пункте может переполнить оперативную память.

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

from implicit.nearest_neighbours import CosineRecommender, tfidf_weight


def run_cosine(X: csr_matrix):
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [16]:
tests.check_cf_recs(metrics_cf)

##### 7.2 TF-IDF

А теперь щепотка магии - с помощью `implicit.nearest_neighbours.tfidf_weight` модифицируем user-item матрицу и получим более высокие метрики.

Все, что нужно -- это применить этот метод к матрице и затем проделать все те же операции, что и раньше (с вычислением похожестей и тд)

In [ ]:
X_tfidf = tfidf_weight(X)
metrics_cf_tfidf = run_cosine(X_tfidf)

metrics_cf_tfidf

In [18]:
tests.check_tfidf_recs(metrics_cf_tfidf)

### 8. ALS (1 балл)

С помощью `implicit.als.AlternatingLeastSquares` над той же самой разреженной user-item матрицей можно обучить ALS, что вам и предлагается сделать.

Чтобы сформировать рекомендации, наша функция `get_candidates_item2item` не нужна -- достаточно использовать метод `model.recommend`.

Нужно обучить ALS с двумя версиями user-item матриц - исходной и tfidf-модифицированной.

In [ ]:
from implicit.als import AlternatingLeastSquares

def run_als(X: csr_matrix, factors: int = 100, reg: float = 0.5, alpha: float = 0.1, iters: int = 20):
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

metrics_als_raw = run_als(X)
metrics_als_tfidf = run_als(X_tfidf)

metrics_als_raw, metrics_als_tfidf

In [20]:
tests.check_als_recs(metrics_als_raw, metrics_als_tfidf)

### 9. Лидерборд и выводы

Собираем таблицу со всеми методами и метриками.

Добавьте 5–10 строк выводов к экспериментам: что работает лучше и почему.


In [ ]:
import pandas as pd

leaderboard = pd.DataFrame([
    {"method": "TopPop", **metrics_toppop},
    {"method": "User-Artist", **metrics_artist},
    {"method": "Item2Item (dataset emb)", **metrics_i2i},
    {"method": "Item2Vec (w2v)", **metrics_w2v},
    {"method": "CF Cosine (raw)", **metrics_cf},
    {"method": "CF Cosine (tf-idf)", **metrics_cf_tfidf},
    {"method": "ALS (raw)", **metrics_als_raw},
    {"method": "ALS (tf-idf)", **metrics_als_tfidf},
])

leaderboard = leaderboard.sort_values(["recall", "ndcg"], ascending=False)
leaderboard

### Вопросы на понимание (1 балл)

1) Почему для кандидатов/метрик важно исключать айтемы, которые пользователь уже видел?  
2) Почему при джойне метаданных (да и вообще почти при любом джойне) нужно использовать how='left', а не how='inner'?
3) В рекомендациях по артистам (с помощью счётчиков) не для всех пользователей может найтись нужное количество кандидатов. Почему?
4) Почему tf-idf улучшает item-based CF? На саму функцию можно посмотреть через `tfidf_weight??`
5) В чем принципиальное отличие между item-to-item методом и методом, при котором мы получаем эмбеддинг пользователя, сложив эмбеддинги его последних взаимодействий, и затем ищем ближайшие эмбеддинги айтемов?
5) Почему ALS выиграл у чистого cosine-item2item?

Ответы - текстом